In [9]:
! pip install google-cloud-aiplatform
! pip install squareup
! pip install langchain

In [4]:
from square.client import Client
import os 

client = Client(
    access_token=os.environ['SQUARE_ACCESS_TOKEN'],
    environment='sandbox')

In [5]:
import json

def get_square_orders():
    result = client.orders.search_orders(
      body = {
        "location_ids": [
           os.environ["SQUARE_LOCATION_ID"]
        ]
      }
    )

    if result.is_success():
      return result.body
    elif result.is_error():
      return result.errors


In [58]:
from langchain.tools import tool
from langchain.chat_models import ChatVertexAI

chat = ChatVertexAI()

In [70]:
from typing import Optional

@tool
def tool_square_count_orders(count_orders= False) -> str:
    '''
    Tool that counts all orders if requested, and count_orders needs to be set to True. By default nothing happens.
    '''
    response = get_square_orders()

    if count_orders:
        return str(len(json.loads(response)['orders']))

    return ""


In [71]:
#response = get_square_orders()

In [60]:
from langchain.agents import initialize_agent, AgentType

tools = [tool_square_count_orders]

agent_chain = initialize_agent(
    tools,
    chat,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

In [61]:
agent_chain("Count all orders")



> Entering new AgentExecutor chain...
 Action:
```
{
  "action": "tool_square_count_orders",
  "action_input": {
    "count_orders": true
  }
}
```


Observation: 3
Thought: Action:
```
{
  "action": "Final Answer",
  "action_input": "There are 3 orders."
}
```



> Finished chain.


{'input': 'Count all orders', 'output': 'There are 3 orders.'}